# Group 2 Capstone Project

#### Import nessecary files

In [55]:
#! pip uninstall scikit-learn --yes
#! pip uninstall imblearn --yes
#! pip install scikit-learn==1.2.2
#! pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)


In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from collections import Counter
import imblearn

#### Read in dataset

https://www.kaggle.com/datasets/willianoliveiragibin/websites-e-comerce

E-commerce data set with columns: 'accessed_date', 'duration_(secs)', 'network_protocol', 'ip', 'bytes', 'accessed_Ffom', 'age', 'gender', 'country', 'membership', 'language','sales', 'returned_amount', 'pay_method' with target variable being 'returned'

In [2]:
#read
df = pd.read_csv("E-commerce Website Logs new.csv")
df.head()

C:\Users\scallejo\AppData\Local\Temp\ipykernel_21964\4241691710.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("E-commerce Website Logs new.csv")


,accessed_date,duration_(secs),network_protocol,ip,bytes,accessed_Ffom,age,gender,country,membership,language,sales,returned,returned_amount,pay_method
0,2017-03-14 17:43:57.172,2533,TCP,1.10.195.126,20100,Chrome,28,Female,CA,Normal,English,261.96,No,0,Credit Card
1,2017-03-14 17:43:57.172,4034,TCP,1.1.217.211,20500,Mozilla Firefox,21,Male,AR,Normal,English,731.94,No,0,Debit Card
2,2017-03-14 17:43:26.135,1525,TCP,1.115.198.107,90100,Mozilla Firefox,20,Male,PL,Normal,English,14.62,No,0,Cash
3,2017-03-14 17:43:26.135,4572,TCP,1.121.152.143,100300,Mozilla Firefox,66,Female,IN,Normal,Spanish,9.575.775,No,0,Credit Card
4,2017-03-14 18:17:09.005,3652,TCP,1.123.135.213,270200,Mozilla Firefox,53,Female,KR,Normal,Spanish,22.368,No,0,Cash


### Data Pre-processing

First, we went into the dataset and understood what each of the columns could contribute to our buisness problem. After thoroughly looking thorugh our options, we decided to drop 'network_protocol', 'ip', 'bytes', 'sales', and 'returned_amount'. The column 'network_protocol' didn't seem like a feature that would provide any useful information to our buisness problem as it does not give insght to any kind of marketing strategy or promotions. The column 'ip' only give a unique value for each data point and that would not be very useful for a model. The column 'bytes' was not defined when we were researching our dataset, thus we were not sure what the values represented whether it be the amount of bytes the customer was using when accessing the website or when purchasing the product, etc. so we deemed it so to remove this column as well. The columns 'sales' and 'returned_amount' are types of target variables that we had the choice of using, but decided against it. Since these columns are directly related to our target variable, wherein a 0 in sales can easily predict that the item was returned and a 0 in returned amount would predict the item is not returned, it would not provide our mode with useful information, thus we removed them.

In [3]:
#start prep
df.drop(['network_protocol', 'ip', 'bytes', 'sales', 'returned_amount'], axis=1, inplace=True)
df.head()

,accessed_date,duration_(secs),accessed_Ffom,age,gender,country,membership,language,returned,pay_method
0,2017-03-14 17:43:57.172,2533,Chrome,28,Female,CA,Normal,English,No,Credit Card
1,2017-03-14 17:43:57.172,4034,Mozilla Firefox,21,Male,AR,Normal,English,No,Debit Card
2,2017-03-14 17:43:26.135,1525,Mozilla Firefox,20,Male,PL,Normal,English,No,Cash
3,2017-03-14 17:43:26.135,4572,Mozilla Firefox,66,Female,IN,Normal,Spanish,No,Credit Card
4,2017-03-14 18:17:09.005,3652,Mozilla Firefox,53,Female,KR,Normal,Spanish,No,Cash


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172838 entries, 0 to 172837
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   accessed_date    172838 non-null  object
 1   duration_(secs)  172838 non-null  int64 
 2   accessed_Ffom    172838 non-null  object
 3   age              172838 non-null  object
 4   gender           172838 non-null  object
 5   country          172838 non-null  object
 6   membership       172838 non-null  object
 7   language         172838 non-null  object
 8   returned         172838 non-null  object
 9   pay_method       172838 non-null  object
dtypes: int64(1), object(9)
memory usage: 13.2+ MB


We can see that there are no null values detected from our initial search of finding them. We wanted to make sure that there were no place holders within each of the columns.

##### 1. Age and Gender column

When looking into the age column, we saw that there were placeholder values that represented null items. For some reason there were 2 kinds of 0s and then dashed lines. However, almost 50% of our data consists of these placeholder values which makes it difficult on deciding what to do with these values. We decided on just removing the those values as it was a way not to skew the data because replacing the mean, median, or mode in those values would just make an our model more biased to that value in the age feature. Additionally, since we still have a decent amount of data to use even after removing those data points, we will tset with the data that was left over and if the metrics point to having a better model with more data point then we may return to these data points and add them in. Furthermore, when researching with this column, we also learned that any unknown genders/ place holder values for the gender column was removed whenever we removed all the unknown ages. Since we are working with the data set without the unknown ages, we can skip identifying any unknown values in the gender column.

In [5]:
df['age'].value_counts(normalize=True)

age
0     0.241648
0     0.182917
--    0.085300
37    0.010247
22    0.010108
43    0.009905
51    0.009882
64    0.009876
24    0.009836
30    0.009824
40    0.009784
48    0.009732
61    0.009685
39    0.009628
27    0.009610
52    0.009604
38    0.009570
19    0.009564
28    0.009552
42    0.009518
41    0.009512
56    0.009494
21    0.009442
67    0.009442
26    0.009425
57    0.009425
62    0.009419
36    0.009408
68    0.009396
46    0.009396
33    0.009379
53    0.009350
69    0.009327
58    0.009321
25    0.009309
29    0.009298
50    0.009292
32    0.009286
34    0.009275
31    0.009263
49    0.009257
66    0.009251
59    0.009251
47    0.009234
23    0.009217
44    0.009217
63    0.009153
60    0.009124
20    0.009072
54    0.009066
45    0.009037
35    0.008985
65    0.008980
55    0.008968
18    0.008939
Name: proportion, dtype: float64

In [6]:
#remove non-integer values
df = df[df['age']!="--"] 

In [7]:
#turn age into integer
df['age'] = df['age'].astype('int64')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158095 entries, 0 to 172837
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   accessed_date    158095 non-null  object
 1   duration_(secs)  158095 non-null  int64 
 2   accessed_Ffom    158095 non-null  object
 3   age              158095 non-null  int64 
 4   gender           158095 non-null  object
 5   country          158095 non-null  object
 6   membership       158095 non-null  object
 7   language         158095 non-null  object
 8   returned         158095 non-null  object
 9   pay_method       158095 non-null  object
dtypes: int64(2), object(8)
memory usage: 13.3+ MB


In [9]:
df['age'].value_counts(normalize=True)

age
0     0.464158
37    0.011202
22    0.011050
43    0.010829
51    0.010804
64    0.010797
24    0.010753
30    0.010740
40    0.010696
48    0.010639
61    0.010589
39    0.010525
27    0.010506
52    0.010500
38    0.010462
19    0.010456
28    0.010443
42    0.010405
41    0.010399
56    0.010380
21    0.010323
67    0.010323
26    0.010304
57    0.010304
62    0.010298
36    0.010285
68    0.010272
46    0.010272
33    0.010253
53    0.010222
69    0.010196
58    0.010190
25    0.010177
29    0.010165
50    0.010158
32    0.010152
34    0.010139
31    0.010127
49    0.010120
59    0.010114
66    0.010114
47    0.010095
23    0.010076
44    0.010076
63    0.010007
60    0.009975
20    0.009918
54    0.009912
45    0.009880
35    0.009823
65    0.009817
55    0.009804
18    0.009773
Name: proportion, dtype: float64

In [10]:
df['gender'].value_counts()

gender
Female     93903
Male       63049
Unknown     1143
Name: count, dtype: int64

In [11]:
#remove placeholders
df = df[df['age']!=0]

In [12]:
df['age'].value_counts()

age
37    1771
22    1747
43    1712
51    1708
64    1707
24    1700
30    1698
40    1691
48    1682
61    1674
39    1664
27    1661
52    1660
38    1654
19    1653
28    1651
42    1645
41    1644
56    1641
67    1632
21    1632
57    1629
26    1629
62    1628
36    1626
46    1624
68    1624
33    1621
53    1616
69    1612
58    1611
25    1609
29    1607
50    1606
32    1605
34    1603
31    1601
49    1600
66    1599
59    1599
47    1596
44    1593
23    1593
63    1582
60    1577
20    1568
54    1567
45    1562
35    1553
65    1552
55    1550
18    1545
Name: count, dtype: int64

## Duplicate Handling

In [15]:
# Combine rows with 'Safari' and 'SafFRi'
df.loc[df['accessed_Ffom'] == 'SafFRi', 'accessed_Ffom'] = 'Safari'

# Check the updated 'accessed_Ffom' column
print("Updated accessed_Ffom:")
print(df['accessed_Ffom'].value_counts())

Updated accessed_Ffom:
accessed_Ffom
Android App        18768
Chrome             13818
Mozilla Firefox    13015
Others             13007
IOS App            10627
Safari              8154
Microsoft Edge      7325
Name: count, dtype: int64


In [16]:
# Capitalizing the first letter of each word
df['language'] = df['language'].str.split().str[0].str.capitalize()
print(df['language'].value_counts())

language
English       49939
Chinese       10172
Spanish        7124
Russian        6013
French         3972
Italian        3643
German          955
Japanese        599
Persian         400
Thai            361
Dutch           288
Portuguese      199
Arabic          186
Polish          123
Mongolian       114
Malayalam        95
Swahili          72
Swedish          71
Nepali           71
Norwegian        58
Slovak           48
Slovene          44
Urdu             39
Romanian         36
Malay            30
Macedonian       24
Serbian          14
Tegulu           13
Marathi          11
Name: count, dtype: int64


## Category Encoding

In [17]:
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])
df['returned'] = le.fit_transform(df['returned'])

c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

In [18]:
encoded_columns = ['accessed_Ffom', 'country', 'membership', 'language', 'pay_method']
ohe = pd.get_dummies(df[encoded_columns],prefix=encoded_columns, drop_first=True)
df_encoded = pd.concat([df, ohe], axis=1)
df_encoded = df_encoded.drop(encoded_columns,axis=1)

In [19]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84714 entries, 0 to 99456
Data columns (total 69 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   accessed_date                  84714 non-null  object
 1   duration_(secs)                84714 non-null  int64 
 2   age                            84714 non-null  int64 
 3   gender                         84714 non-null  int32 
 4   returned                       84714 non-null  int32 
 5   accessed_Ffom_Chrome           84714 non-null  bool  
 6   accessed_Ffom_IOS App          84714 non-null  bool  
 7   accessed_Ffom_Microsoft Edge   84714 non-null  bool  
 8   accessed_Ffom_Mozilla Firefox  84714 non-null  bool  
 9   accessed_Ffom_Others           84714 non-null  bool  
 10  accessed_Ffom_Safari           84714 non-null  bool  
 11  country_AR                     84714 non-null  bool  
 12  country_AT                     84714 non-null  bool  
 13  countr

## Undersampling and Oversampling for Imbalanced Data

In [20]:
X = df_encoded.drop(["returned","accessed_date","duration_(secs)"], axis=1)
y = df_encoded.returned
print(Counter(y))

Counter({0: 72341, 1: 12373})


In [21]:
from imblearn.over_sampling import RandomOverSampler

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority') #This means that if the majority class had 1,000 examples and the minority class had 100, 
                                                             # this strategy would oversampling the minority class so that it has 1,000 examples.

X_over, y_over = oversample.fit_resample(X, y)
print(Counter(y_over))

c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

Counter({0: 72341, 1: 72341})


In [22]:
from imblearn.under_sampling import RandomUnderSampler

# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
# fit and apply the transform
X_under, y_under = undersample.fit_resample(X, y)
# summarize class distribution
print(Counter(y_under))

Counter({0: 12373, 1: 12373})


c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\scallejo\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

## Train Test Split

In [24]:
from sklearn.model_selection import train_test_split

Xo_train, Xo_test, yo_train, yo_test = train_test_split(X_over, y_over, test_size=0.25, random_state=42)
Xu_train, Xu_test, yu_train, yu_test = train_test_split(X_under, y_under, test_size=0.25, random_state=42)

## Model 2: Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

#### Oversampling

In [29]:
clf = RandomForestClassifier(random_state=0)
clf.fit(Xo_train, yo_train)

RandomForestClassifier(random_state=0)

In [30]:
clf.score(Xo_train,yo_train)

0.9490927187105455

In [31]:
clf.score(Xo_test,yo_test)

0.8842995770092063

#### Undersampling

In [32]:
clf = RandomForestClassifier(random_state=0)
clf.fit(Xu_train, yu_train)

RandomForestClassifier(random_state=0)

In [33]:
clf.score(Xu_train,yu_train)

0.960773748585592

In [34]:
clf.score(Xu_test,yu_test)

0.5134960400840471

There is possible overfitting case here due to tht testing scorea being lower and the training scores being so good

#### oversampling

In [35]:
param_grid = { 
    'n_estimators': [25, 50, 100, 150], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [3, 6, 9] 
} 

In [41]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(RandomForestClassifier(random_state=0), param_grid=param_grid) 
grid_search.fit(Xo_train, yo_train) 
print(grid_search.best_estimator_) 

RandomForestClassifier(max_depth=9, max_features=None, n_estimators=50,
                       random_state=0)


In [56]:

clf_grid = RandomForestClassifier(max_depth=25) #max_features= None, n_estimators=25
clf_grid.fit(Xo_train, yo_train) 
clf_grid.score(Xo_train, yo_train)

0.9380892259770899

In [57]:
clf_grid.score(Xo_test, yo_test)

0.8681816925161041

#### undersampling

In [62]:
clf_grid = RandomForestClassifier(max_depth=20) #max_features= None, n_estimators=25
clf_grid.fit(Xu_train, yu_train) 
clf_grid.score(Xu_train, yu_train)

0.9010722560482785

In [63]:
clf_grid.score(Xu_test, yu_test)

0.5170518829804429

Seems as though hyperparameter tuning does not help our model... but it looks like over sampling is better for our data.